In [5]:
import aiohttp
import asyncio
import ujson
from tabulate import tabulate
from copy import deepcopy
from json import dumps,loads

# Pairs which generate orderbook for.
PAIRS = [
        'BTCUSD',
        #'ETCBTC',
        # 'ETCUSD',
        # 'ETHBTC',
        # 'ETHUSD',
        # 'XMRBTC',
        # 'XMRUSD',
        # 'ZECBTC',
        # 'ZECUSD'
    ]

# If there is n pairs we need to subscribe to n websocket channels.
# This the subscription message template.
# For details about settings refer to https://bitfinex.readme.io/v2/reference#ws-public-order-books.
GDAX_MESSAGE = {
                "type": "subscribe",
                "product_ids": PAIRS,
                "channels": ["ticker"]
               }

SUB_MESG = {
        'event': 'subscribe',
        'channel': 'book',
        'freq': 'F1',
        'len': '25',
        'prec': 'P0'
        # 'pair': <pair>
    }

def GetPrices(updateMessage, Pair):
    data = ujson.loads(res.data)
    



def build_book(res, pair):
    """ Updates orderbook.
    :param res: Orderbook update message.
    :param pair: Updated pair.
    
    """

    # Filter out subscription status messages.
    if res.data[0] == '[':

        # String to json
        data = ujson.loads(res.data)[1]

        # Build orderbook
        # Observe the structure of orderbook. The prices are keys for corresponding count and amount.
        # Structuring data in this way significantly simplifies orderbook updates.
        if len(data) > 10:
            bids = {
                       str(level[0]): [str(level[1]), str(level[2])]
                       for level in data if level[2] > 0
            }

            asks = {
                       str(level[0]): [str(level[1]), str(level[2])[1:]]
                       for level in data if level[2] < 0
            }

            orderbooks[pair]['bids'] = bids
            orderbooks[pair]['asks'] = asks

        # Update orderbook and filter out heartbeat messages.
        elif data[0] != 'h':

            # Example update message structure [1765.2, 0, 1] where we have [price, count, amount].
            # Update algorithm pseudocode from Bitfinex documentation:
            # 1. - When count > 0 then you have to add or update the price level.
            #   1.1- If amount > 0 then add/update bids.
            #   1.2- If amount < 0 then add/update asks.
            # 2. - When count = 0 then you have to delete the price level.
            #   2.1- If amount = 1 then remove from bids
            #   2.2- If amount = -1 then remove from asks

            data = [str(data[0]), str(data[1]), str(data[2])]
            if int(data[1]) > 0:  # 1.

                if float(data[2]) > 0:  # 1.1
                    orderbooks[pair]['bids'].update({data[0]: [data[1], data[2]]})

                elif float(data[2]) < 0:  # 1.2
                    orderbooks[pair]['asks'].update({data[0]: [data[1], str(data[2])[1:]]})

            elif data[1] == '0':  # 2.

                if data[2] == '1':  # 2.1
                    if orderbooks[pair]['bids'].get(data[0]):
                        del orderbooks[pair]['bids'][data[0]]

                elif data[2] == '-1':  # 2.2
                    if orderbooks[pair]['asks'].get(data[0]):
                        del orderbooks[pair]['asks'][data[0]]

async def print_books():
    """ Prints orderbooks snapshots for all pairs every 10 seconds. """
    while True:
        await asyncio.sleep(10)
        for pair in PAIRS:
            bids = [[v[1], v[0], k] for k, v in orderbooks[pair]['bids'].items()]
            asks = [[k, v[0], v[1]] for k, v in orderbooks[pair]['asks'].items()]
            bids.sort(key=lambda x: float(x[2]), reverse=True)
            asks.sort(key=lambda x: float(x[0]))
            table = [[*bid, *ask] for (bid, ask) in zip(bids, asks)]
            headers = ['bid:amount', 'bid:count', 'bid:price', 'ask:price', 'ask:count', 'ask:amount']
            print('orderbook for {}'.format(pair))
            print(tabulate(table, headers=headers))


async def get_book(pair, session):
    """ Subscribes for orderbook updates and fetches updates. """
    print('enter get_book, pair: {}'.format(pair))
    pair_dict = deepcopy(SUB_MESG)
    pair_dict.update({'pair': pair})
    async with session.ws_connect('wss://api.bitfinex.com/ws/2') as ws:
        ws.send_json(pair_dict)
        while 1:
            res = await ws.receive()
            print(pair_dict['pair'], res.data)  # debug
            build_book(res, pair)

async def main():
    """ Driver coroutine. """
    async with aiohttp.ClientSession() as session:
        coros = [
            get_book(pair, session)
            for pair in PAIRS
        ]
        # Append coroutine for printing orderbook snapshots every 10s.
        coros.append(print_books())

        await asyncio.wait(coros)


In [6]:
"""

On line 127 
The rest of main() coro just 
"""

async def main():
    """ Driver coroutine. """
   #The main() coro initializes an aiohttp session which we will use for all WS connections in the script. 
    async with aiohttp.ClientSession() as session:
        # create list of get_book() coroutines already equipped with arguments. 
        # Append coroutine for printing orderbook snapshots every 10s.
        coros = [
            get_book(pair, session)
            for pair in PAIRS
        ]
       #appends printing coro to the coros list and registers the content of coros to the event loop. 
        coros.append(print_books())
        # the control is switched back to the event loop because await asyncio.wait(coros) is blocking call.
        await asyncio.wait(coros)

In [7]:
orderbooks = {
    pair: {}
    for pair in PAIRS
}
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

enter get_book, pair: BTCUSD
BTCUSD []
BTCUSD {"event":"info","version":2}
BTCUSD {"event":"subscribed","channel":"book","chanId":496463,"symbol":"tBTCUSD","prec":"P0","freq":"F1","len":"25","pair":"BTCUSD"}
BTCUSD [496463,[[5513,1,13.8045],[5511.9,1,0.5],[5510.9,1,0.5],[5510.2,1,0.070575],[5510.1,1,1.807],[5510,1,0.5],[5509.2,1,0.0509],[5509.1,1,12.5991],[5508.7,1,1],[5508,2,2.22989999],[5507.7,1,0.7514],[5507.6,1,1],[5507.4,1,0.08467],[5507,1,100],[5506.8,1,0.00673168],[5506.4,1,4.8],[5502.2,1,0.09130453],[5502.1,1,0.2795957],[5502,1,0.2],[5501.8,1,0.24530931],[5501.5,1,7.75335637],[5501.4,1,10.835946],[5501,19,10.74637251],[5500.1,1,4.8],[5500,1,2.7096],[5514.4,1,-5.83722632],[5514.6,1,-1],[5514.8,1,-10.9914],[5515,1,-0.16920362],[5516.6,1,-1],[5517,1,-0.11],[5517.2,1,-0.2],[5517.3,1,-10.43],[5518.3,2,-0.90061287],[5518.5,1,-1],[5518.7,1,-5],[5519,1,-0.2],[5519.1,2,-0.50894725],[5519.3,1,-0.006],[5519.5,1,-0.01728097],[5519.6,1,-0.3],[5519.7,1,-0.006],[5520,8,-8.922],[5520.2,2,-9.88

KeyboardInterrupt: 

# MAIN 

In [1]:
import aiohttp
import asyncio
import ujson
from tabulate import tabulate
from copy import deepcopy
PAIRS = [
        'BTCUSD'
        #'ETCBTC',
        # 'ETCUSD',
        # 'ETHBTC',
        # 'ETHUSD',
        # 'XMRBTC',
        # 'XMRUSD',
        # 'ZECBTC',
        # 'ZECUSD'
    ]
GDAX_MESSAGE = {
                "type": "subscribe",
                "product_ids": PAIRS,
                "channels": ["ticker"]
               }
def UpdatePrices(updateMessage, pair):
    data = ujson.loads(updateMessage.data)
    print(data)
    Prices[pair]= data["price"]
    
async def SubscribePrices(pair, session):
    """ Subscribes for orderbook updates and fetches updates. """
    print('enter get_book, pair: {}'.format(pair))
    message = deepcopy(GDAX_MESSAGE)
    async with session.ws_connect("wss://ws-feed.gdax.com") as ws:
        ws.send_json(dumps(GDAX_MESSAGE))
        while True:
            res = await ws.receive()
          
            print(json.loads(res.data)) # debug

            UpdatePrices(res, pair)
            
           
            
async def printPrices():

    while True:
        await asyncio.sleep(10)
        for pair in PAIRS:
            print(pair,Prices[pair])
            
            

In [2]:
async def main():
    """ Driver coroutine. """
   #The main() coro initializes an aiohttp session which we will use for all WS connections in the script. 
    async with aiohttp.ClientSession() as session:
        # create list of get_book() coroutines already equipped with arguments. 
        # Append coroutine for printing orderbook snapshots every 10s.
        coros = [
            SubscribePrices(pair, session)
            for pair in PAIRS
        ]
       #appends printing coro to the coros list and registers the content of coros to the event loop. 
        coros.append(printPrices())
        # the control is switched back to the event loop because await asyncio.wait(coros) is blocking call.
        await asyncio.wait(coros)

In [3]:
if __name__ =="__main__":
    Prices = {
        pair:[] 
        for pair in PAIRS
    }
    #we create and start asyncio event loop which in turn executes main() coroutine
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

enter get_book, pair: BTCUSD
BTCUSD []


KeyboardInterrupt: 

In [10]:
GDAX_MESSAGE = {
                "type": "subscribe",
                "product_ids": PAIRS,
                "channels": ["ticker"]
               }
import json

In [15]:

json.dumps(GDAX_MESSAGE)

'{"type": "subscribe", "product_ids": ["BTCUSD"], "channels": ["ticker"]}'